In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error

In [21]:
df = pd.read_csv('../data/train_data_encoded_full_1.csv')
df['weight'] = df['weight'] * 0.2
df.drop(['weight', 'depth_percent', 'table_percent'], axis=1, inplace=True)
print('shape:', df.shape)
df.head(5)

shape: (36960, 5)


,quality,color,clarity,price,volume
0,3,6,1,1.848927,1.694056
1,4,5,5,2.217354,2.224149
2,4,4,3,2.077314,2.064551
3,2,3,1,2.124773,2.243388
4,3,3,4,1.885250,1.761473


In [22]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print('\n')
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    29568.000000
mean         2.033940
std          0.127191
min          1.755614
25%          1.918833
50%          2.041090
75%          2.141007
max          2.286659
Name: price, dtype: float64


Datos de testeo
-----------------------
count    7392.000000
mean        2.031700
std         0.128554
min         1.760785
25%         1.914272
50%         2.037056
75%         2.140654
max         2.286659
Name: price, dtype: float64


In [23]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [24]:
n_estimators = [int(x) for x in np.linspace(10,200,10)]
max_depth = [int(x) for x in np.linspace(10,100,10)]
min_samples_split = [2,3,4,5,10]
min_samples_leaf = [1,2,4,10,15,20]
random_grid = {'n_estimators':n_estimators,'max_depth':max_depth,
               'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf}

random_grid

{'n_estimators': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'min_samples_split': [2, 3, 4, 5, 10],
 'min_samples_leaf': [1, 2, 4, 10, 15, 20]}

In [25]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               cv = 3)

rf_random.fit(X_train,y_train)
y_pred = rf_random.predict(X_test)

print("RMSE: {}".format(np.sqrt(mean_squared_error((y_test),(y_pred)))))
print("R2  : {}".format(np.sqrt(r2_score((y_test),(y_pred)))))
rf_random.best_params_

RMSE: 0.015063564933709193
R2  : 0.9931101485979885


{'n_estimators': 178,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_depth': 50}

In [19]:
rf = RandomForestRegressor(n_estimators=157,
                         min_samples_split=2,
                         min_samples_leaf=4,
                         max_depth=30)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

print("RMSE: {}".format(np.sqrt(mean_squared_error((y_test),(y_pred)))))
print("R2  : {}".format(np.sqrt(r2_score((y_test),(y_pred)))))

RMSE: 0.015400821107328384
R2  : 0.9927970515188951
